# Econ 101b Problem Set 5

#### Probem set will be due Fri Sep 21 at midnight...

#### Sections between Tu AM & Th AM lecture will be problem set section...

----

&nbsp;

## Setting up the Python/Jupyter environment

In [ ]:
%%javascript

IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;}

In [ ]:
# ABOVE CELL IS "NO SCROLLING SUBWINDOWS" SETUP
#
# keep output cells from shifting to autoscroll: little scrolling
# subwindows within the notebook are an annoyance...

In [ ]:
# THIS CELL LOADS THE LIBRARIES
#
# set up the environment by reading in every library we might need: 
# os... graphics... data manipulation... time... math... statistics...

import sys
import os
from urllib.request import urlretrieve

import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import Image

import pandas as pd
from pandas import DataFrame, Series
from datetime import datetime

import scipy as sp
import numpy as np
import math
import random

import seaborn as sns
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# PRETTIER GRAPHICS SETUP
#
# graphics setup: seaborn-whitegrid and figure size;
# graphs in the notebook itself...

%matplotlib inline 

plt.style.use('seaborn-whitegrid')

figure_size = plt.rcParams["figure.figsize"]
figure_size[0] = 12
figure_size[1] = 10
plt.rcParams["figure.figsize"] = figure_size

In [ ]:
# THIS CELL IS THE KEY TO THE OKPY.ORG AUTOGRADER SYSTEM
#
# Don't change this cell; just run it. 
# The result will give you directions about how to log in to the submission system, called OK.
# Once you're logged in, you can run this cell again, but it won't ask you who you are because
# it remembers you. However, you will need to log in once per assignment.

!pip install -U okpy

from client.api.notebook import Notebook

ok = Notebook('ps5.ok')
_ = ok.auth(force=True, inline=True)

### ⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎⬆︎
### RUN THESE CELLS ABOVE FIRST

# Problem Set 5 (Due Sep 21 11:59:59 PM)

----

**(Task A) Graphing the Production Function**

The parameter $ \alpha $ (or alpha) in the Solow Growth Model governs the economy's degree to which growth is oriented toward capital and investment being important (a high value of $ 
\alpha $) or not so important (a low value of $ \alpha $). This task aims to start familiarizing you with its role:

The code cell below "_TASK A CODE_" calculates and graphs the per-worker version of the production function:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $ \frac{Y}{L} = \left(\frac{K}{L}\right)^{\alpha}E^{1-\alpha}  $

for $ \alpha $ = 0.5, E = 1. 

Copy and paste it further below. Then alter it to calculate the production function for $ \alpha $ = 0.25, E = 2, and set the output variable:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; OPW_quartertwo = output_per_worker[500, 1]. Run it. 

In the subsequent markdown cell "**(Task A) Answer Discussion**", describe the these two cases of the production function. How are the economies they describe alike? How are the economies they describe different?

In [ ]:
# TASK A CODE

alpha = 0.5
E = 1

output_per_worker = np.zeros((501, 2))

for i in range(501):
    lowestkoverl = 0
    output_per_worker[i, 0] = lowestkoverl + 2 * i
    output_per_worker[i, 1] = (output_per_worker[i, 0] ** alpha) * E
output_per_worker_df = DataFrame(data = output_per_worker, 
    columns = ["Capital_per_Worker", "Output_per_Worker"])

output_per_worker_df.set_index('Capital_per_Worker').plot()
plt.xlabel("Capital per Worker")
plt.ylabel("Output per Worker")
plt.ylim(0, )

str1 = "Production Function:"
str2 = "Output per Worker as a Function of Capital per Worker"
str3 = "for a Constant Value of the Efficiency of Labor"
str4 = "α = 0.5, E = 1"
plt.title(str1 + "\n" + str2 + "\n" + str3 + "\n" + str4, size = 20)

OPW_halfone = output_per_worker[500, 1]

print("Output per worker =", OPW_halfone)

<font color="blue"> **(Task A) Answer Discussion:** 
    
</font>

In [ ]:
ok.grade('q03')

----

&nbsp;

**(Task B): Plotting the Cobb-Douglas Production Function**:

Continuing this familiarization process, we will now consider five values for this parameter $ \alpha $: 1/4, 1/3, 1/2, 2/3, and 3/4:

The code cell below, "_TASK B CODE_", when run plots output per worker for five different values of the orientation-of-growth-toward-capital-and-investment production function parameter α, adjusting E for each curve so that the value of output per worker is 40 when the capital-output ratio is 200:

1. Run the cell.
2. Copy the code in the cell. Paste it in a new cell below "_TASK B CODE_". Alter the code so that it plots the production function for the five different production function parameters, but with the value of output per worker equal to 200 whenever capital per worker is 800.
3. Then in the "**(Task B) Answer Discussion**" cell, provide your assessment of whether this production function is in fact sophisticated enough: In your view, does the power to vary $ \alpha $ and E allow an economist to come "close enough" to modeling the different relationships between capital per worker and output per worker that different economies might find themselves having?

In [ ]:
# TASK B CODE 

import numpy as np
from pandas import DataFrame, Series
import matplotlib as mpl
import matplotlib.pyplot as plt

E = 40

for a in range(5):
    output_per_worker = np.zeros((501, 2))
    alpha = 0.15 * (a + 1)
    for i in range(501):
        output_per_worker[i, 0] = 2 * i
        output_per_worker[i, 1] = ((output_per_worker[i, 0] ** alpha) * E / 
            ((200) ** alpha) )
    output_per_worker_df = DataFrame(data = output_per_worker, 
        columns = ["Capital_per_Worker", "Output_per_Worker"])

    plt.plot(output_per_worker_df.Capital_per_Worker, output_per_worker_df.Output_per_Worker)
    plt.legend(["α=0.15", "α=0.30", "α=0.45", "α=0.60", "α=0.75"])
    
plt.xlabel("Capital per Worker")
plt.ylabel("Output per Worker")
plt.title("Production Function for Varying α\nHolding f(200) = 40", size = 24)
plt.xlim(0, 1000)
plt.ylim(0, )

<font color="blue"> **(Task B) Answer Discussion:** 
    
</font>

In [ ]:
ok.grade('q04')

**(Task C) Getting a Sense of How the Solow Growth Model Behaves Via Simulation**

Moving on to dynamics—not just what the production function looks like at a moment in time, but how a Solow Growth Model economy behaves over time:

We start out with us giving you a function:

    sgm_3_bgp_100yr_run(L0, E0, n=0.01, g=0.015, s=0.20,
    alpha=0.5, delta=0.025, Delta_s=0, Delta_g=0, Delta_n=0, 
    T = 100, graphs = "LEVELS")

calculating and plotting levels of Solow growth model variables.
    
Required arguments for the function are the initial level of the labor force, L0, the first argument of the function, and the initial level of the efficiency of labor, E0, the second argument of the function.

Optional arguments include:

* the curvature of the production function parameter alpha (or $ \alpha $), 

* the depreciation rate delta (or $ \delta $), and the time T for which the simulation is to be run; 

plus:

* the value n of the labor-force growth rate for the baseline balanced-growth path that the economy is initially on, and the value Delta_n (or $ {\Delta}n $) for the differential in the labor-force growth rate between the initial and the alternative balanced-gorwth path;

* the value g of the efficiency-of-labor growth rate for the baseline balanced-growth path that the economy is on, and the value Delta_g (or $ {\Delta}g $) of the differential between the baseline and the alternative scenario;   

* the value s of the savings-investment share of output Y for the baseline balanced-growth path that the economy is on, and the value Delta_s (or $ {\Delta}s $) of the differential between the baseline and the alternative scenario;

* the period over which the simulation is to be run

* whether the function is to plot the variables in levels (graph = "LEVELS"), in logs (graph = "LOGS"), or not plot them at all (graph = "NONE")

If optional arguments are not given, then they take on their default values:

* Zero for three baseline-alternative scenario differentials: Delta_s, Delta_n, and Delta_g ($ {\Delta}s, {\Delta}n, {\Delta}g) $.
* A labor-force growth rate n=0.01
* An efficiency-of-labor growth rate g=0.015
* A savings-investment share s=0.20
* An orientation-of-growth-toward capital parameter: ($ \alpha $) alpha=0.5
* A depreciation rate of capital ($ \delta $) delta=0.025
* A time span for the simulation to run T = 100
* A final flag for the type of graphs to be plotted: graphs == "LEVELS" leads the function to plot the levels of the SGM variables.

The function will return a dictionary. If you save the return and park it in a variable, you can then access all of the caculated values inside the function in the "df" lobe, and all the graphics in the "plots" lobe.

Begin by checking the function: run the base case with the two percentage point boost to annual labor force growth Delta_n = 0.01 ($ {\Delta}n = 0.01 $), run other cases, and report any bugs you think you find—screenshots and emails to delong@econ.berkeley.edu.

Then go on to the descriptions and tasks further down...

In [ ]:
# TASK C CODE
#
# FUNCTION FOR CALCULATING AND GRAPHING THE LEVELS OF 
# SOLOW GROWTH MODEL VARIABLES IN SIMULATIONS
#
# might as well put "check that common libraries are active" as a default header
# in every long python code cell...

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import numpy as np
%matplotlib inline

# we are going to want to see what happens for lots of
# different model parameter values and base conditions,
# so stuff our small simulation program inside a function, so 
# we can then invoke it with a single line...
#
# we are going to assume the economy starts on its base
# balanced growth path...
#
# we are going to want to keep track not just of what the
# economy's variables are at each point in time, but also 
# what the base and alternative balanced-growth path 
# values of variables are. Given the parameters, the new BGP 
# is attracting the economy to it at the speed (1-α)(n+g+δ), 
# closing that fraction of the gap between its current state 
# and the balanced growth path attractor every period...
#
# variables are:
#
# * L0: initial labor force
# * E0: initial efficiency of labor
# * n: labor-force growth rate
# * g: efficiency-of-labor growth rate
# * s: savings-investment share
# * alpha: growth orientation toward capital and investment parameter
# * delta: depreciation rate
# * Delta_s: jump in s 
# * Delta_g: jump in g
# * Delta_n: jump in n


def sgm_3_bgp_100yr_run(L0, E0, n=0.01, g=0.015, s=0.20, 
    alpha=0.5, delta=0.025, Delta_s=0, Delta_g=0, Delta_n=0, 
    T = 100, graphs="LEVELS"):

    sg_df = pd.DataFrame(index=range(T),columns=['Labor', 
        'Efficiency',
        'Capital',
        'Output',
        'Output_per_Worker',
        'Capital_Output_Ratio',
        'BGP_base_Labor',
        'BGP_base_Efficiency',
        'BGP_base_Output',
        'BGP_base_Output_per_Worker',
        'BGP_base_Capital_Output_Ratio',
        'BGP_base_Capital',
        'BGP_alt_Labor',
        'BGP_alt_Efficiency',
        'BGP_alt_Output',
        'BGP_alt_Output_per_Worker',
        'BGP_alt_Capital_Output_Ratio',
        'BGP_alt_Capital'],
        dtype='float')

    sg_df.Labor[0] = L0
    sg_df.BGP_base_Labor[0] = L0
    sg_df.BGP_alt_Labor[0] = L0
    sg_df.Efficiency[0] = E0
    sg_df.BGP_base_Efficiency[0] = E0
    sg_df.BGP_alt_Efficiency[0] = E0

    KoverY_base_steady_state = s/(n+g+delta)
    YoverL_base_steady_state = ((s/(n+g+delta))**(alpha/(1-alpha)) 
        * E0)
    KoverL_base_steady_state = (YoverL_base_steady_state *
        KoverY_base_steady_state)
    
    sg_df.Capital[0] = KoverL_base_steady_state * L0
    sg_df.Output[0] = (sg_df.Capital[0]**alpha * (sg_df.Labor[0] * 
        sg_df.Efficiency[0])**(1-alpha))
    sg_df.Output_per_Worker[0] = sg_df.Output[0]/sg_df.Labor[0]
    sg_df.Capital_Output_Ratio[0] = sg_df.Capital[0]/sg_df.Output[0]
    
    sg_df.BGP_base_Capital_Output_Ratio[0] = (s / (n + g + delta))
    sg_df.BGP_base_Output_per_Worker[0] = sg_df.Efficiency[0] * (
        sg_df.BGP_base_Capital_Output_Ratio[0]*(alpha/(1 - alpha)))
    sg_df.BGP_base_Output[0] = sg_df.BGP_base_Output_per_Worker[0] * sg_df.Labor[0]
    sg_df.BGP_base_Capital[0] = sg_df.BGP_base_Output[0] * (
        sg_df.BGP_base_Capital_Output_Ratio[0])
    
    sg_df.BGP_alt_Capital_Output_Ratio[0] = ((s + Delta_s) / 
        (n + Delta_n + g + Delta_g + delta))
    sg_df.BGP_alt_Output_per_Worker[0] = sg_df.Efficiency[0] * (
        sg_df.BGP_alt_Capital_Output_Ratio[0]*(alpha/(1 - alpha)))
    sg_df.BGP_alt_Output[0] = sg_df.BGP_alt_Output_per_Worker[0] * sg_df.Labor[0]
    sg_df.BGP_alt_Capital[0] = sg_df.BGP_alt_Output[0] * (
        sg_df.BGP_alt_Capital_Output_Ratio[0])
    
    for i in range(T):
        sg_df.Labor[i+1] = (sg_df.Labor[i] * np.exp(n + Delta_n))
        sg_df.Efficiency[i+1] = (sg_df.Efficiency[i] * np.exp(g + Delta_g))
        KoverY_current = sg_df.Capital[i]/sg_df.Output[i]
        sg_df.Capital[i+1] = (sg_df.Capital[i] * np.exp((s+Delta_s)/ 
            KoverY_current - delta))
        sg_df.Output[i+1] = (sg_df.Capital[i+1]**alpha * 
            (sg_df.Labor[i+1] * sg_df.Efficiency[i+1])**(1-alpha))
        sg_df.Output_per_Worker[i+1] = sg_df.Output[i+1]/sg_df.Labor[i+1]
        sg_df.Capital_Output_Ratio[i+1] = (sg_df.Capital[i+1]/
            sg_df.Output[i+1])

    for i in range(T):
        sg_df.BGP_base_Labor[i+1] = (sg_df.BGP_base_Labor[i] * np.exp(n))
        sg_df.BGP_base_Efficiency[i+1] = (sg_df.BGP_base_Efficiency[i] * np.exp(g))
        sg_df.BGP_base_Capital_Output_Ratio[i+1] = (s / (n + g + delta))
        sg_df.BGP_base_Output_per_Worker[i+1] = sg_df.BGP_base_Efficiency[i+1] * (
            sg_df.BGP_base_Capital_Output_Ratio[i+1]**(alpha/(1 - alpha)))
        sg_df.BGP_base_Output[i+1] = (sg_df.BGP_base_Output_per_Worker[i+1] * 
            sg_df.BGP_base_Labor[i+1])
        sg_df.BGP_base_Capital[i+1] = (s / (n + g + delta))**(1/(1-alpha)) * (
            sg_df.Efficiency[i+1] * sg_df.Labor[i+1])

    for i in range(T):
        sg_df.BGP_alt_Labor[i+1] = (sg_df.BGP_alt_Labor[i] * np.exp(n + Delta_n))
        sg_df.BGP_alt_Efficiency[i+1] = (sg_df.BGP_alt_Efficiency[i] * np.exp(g+Delta_g))
        sg_df.BGP_alt_Capital_Output_Ratio[i+1] = ((s+ Delta_s) / 
            (n + Delta_n + g + Delta_g + delta))
        sg_df.BGP_alt_Output_per_Worker[i+1] = sg_df.BGP_alt_Efficiency[i+1] * (
            sg_df.BGP_alt_Capital_Output_Ratio[i+1]**(alpha/(1 - alpha)))
        sg_df.BGP_alt_Output[i+1] = (sg_df.BGP_alt_Output_per_Worker[i+1] * 
            sg_df.BGP_alt_Labor[i+1])
        sg_df.BGP_alt_Capital[i+1] = ((s + Delta_s) / (n + Delta_n + g + Delta_g + delta))**(1/(1-alpha)) * (
            sg_df.BGP_alt_Efficiency[i+1] * sg_df.BGP_alt_Labor[i+1])  
    
    if (graphs == "LEVELS"):
        fig = plt.figure(figsize=(12, 12))

        ax1 = plt.subplot(2,3,1)
        sg_df.BGP_base_Labor.plot(ax = ax1, title = "BGP (base) Labor")
        sg_df.BGP_alt_Labor.plot(ax = ax1, title = "BGP (alt) Labor")
        sg_df.Labor.plot(ax = ax1, title = "Labor Force")
        plt.ylabel("Values")
        plt.ylim(0, )

        ax2 = plt.subplot(2,3,2)
        sg_df.BGP_base_Efficiency.plot(ax = ax2, title = "BGP (base) Efficiency")
        sg_df.BGP_alt_Efficiency.plot(ax = ax2, title = "BGP (alt) Efficiency")
        sg_df.Efficiency.plot(ax = ax2, title = "Efficiency of Labor")
        plt.ylim(0, )
    
        ax3 = plt.subplot(2,3,3)
        sg_df.BGP_base_Capital.plot(ax = ax3, title = "BGP (base) Capital Stock")
        sg_df.BGP_alt_Capital.plot(ax = ax3, title = "BGP (alt) Capital Stock")
        sg_df.Capital.plot(ax = ax3, title = "Capital Stock")
        plt.ylim(0, )

        ax4 = plt.subplot(2,3,4)
        sg_df.BGP_base_Output.plot(ax = ax4, title = "BGP (base) Output")
        sg_df.BGP_alt_Output.plot(ax = ax4, title = "BGP (alt) Output")
        sg_df.Output.plot(ax = ax4, title = "Output")
        plt.ylabel("Values")
        plt.xlabel("Years")
        plt.ylim(0, )

        ax5 = plt.subplot(2,3,5)
        sg_df.BGP_base_Output_per_Worker.plot(ax = ax5, title = "BGP (base) Output per Worker")
        sg_df.BGP_alt_Output_per_Worker.plot(ax = ax5, title = "BGP (alt) Output per Worker")
        sg_df.Output_per_Worker.plot(ax = ax5, title = "Output per Worker")
        plt.xlabel("Years")
        plt.ylim(0, )

        ax6 = plt.subplot(2,3,6)
        sg_df.BGP_base_Capital_Output_Ratio.plot(ax = ax6, 
            title = "BGP (base) Capital-Output Ratio")
        sg_df.BGP_alt_Capital_Output_Ratio.plot(ax = ax6, 
            title = "BGP (alt) Capital-Output Ratio")
        sg_df.Capital_Output_Ratio.plot(ax = ax6, 
            title = "Capital-Output Ratio")
        plt.xlabel("Years")
        plt.ylim(0, )

        plt.suptitle('Solow Growth Model: Levels: Simulation Run', size = 20)

        plt.show()
        
    if (graphs == "LOGS"):
        fig = plt.figure(figsize=(12, 12))

        ax1 = plt.subplot(2,3,1)
        np.log(sg_df.BGP_base_Labor).plot(ax = ax1, title = "BGP (base) Labor")
        np.log(sg_df.BGP_alt_Labor).plot(ax = ax1, title = "BGP (alt) Labor")
        np.log(sg_df.Labor).plot(ax = ax1, title = "Log Labor Force")
        plt.ylabel("Values")
        plt.ylim(0, )

        ax2 = plt.subplot(2,3,2)
        np.log(sg_df.BGP_base_Efficiency).plot(ax = ax2, title = "BGP (base) Efficiency")
        np.log(sg_df.BGP_alt_Efficiency).plot(ax = ax2, title = "BGP (alt) Efficiency")
        np.log(sg_df.Efficiency).plot(ax = ax2, title = "Log Efficiency of Labor")
        plt.ylim(0, )
    
        ax3 = plt.subplot(2,3,3)
        np.log(sg_df.BGP_base_Capital).plot(ax = ax3, title = "BGP (base) Capital Stock")
        np.log(sg_df.BGP_alt_Capital).plot(ax = ax3, title = "BGP (alt) Capital Stock")
        np.log(sg_df.Capital).plot(ax = ax3, title = "Log Capital Stock")
        plt.ylim(0, )

        ax4 = plt.subplot(2,3,4)
        np.log(sg_df.BGP_base_Output).plot(ax = ax4, title = "BGP (base) Output")
        np.log(sg_df.BGP_alt_Output).plot(ax = ax4, title = "BGP (alt) Output")
        np.log(sg_df.Output).plot(ax = ax4, title = "Log Output")
        plt.ylabel("Values")
        plt.xlabel("Years")
        plt.ylim(0, )

        ax5 = plt.subplot(2,3,5)
        np.log(sg_df.BGP_base_Output_per_Worker).plot(ax = ax5, title = "BGP (base) Output per Worker")
        np.log(sg_df.BGP_alt_Output_per_Worker).plot(ax = ax5, title = "BGP (alt) Output per Worker")
        np.log(sg_df.Output_per_Worker).plot(ax = ax5, title = "Log Output per Worker")
        plt.xlabel("Years")
        plt.ylim(0, )

        ax6 = plt.subplot(2,3,6)
        sg_df.BGP_base_Capital_Output_Ratio.plot(ax = ax6, 
            title = "BGP (base) Capital-Output Ratio")
        sg_df.BGP_alt_Capital_Output_Ratio.plot(ax = ax6, 
            title = "BGP (alt) Capital-Output Ratio")
        sg_df.Capital_Output_Ratio.plot(ax = ax6, 
            title = "Capital-Output Ratio")
        plt.xlabel("Years")
        plt.ylim(0, )

        plt.suptitle('Solow Growth Model: Logs: Simulation Run', size = 20)

        plt.show()
    
    if ((graphs != "LEVELS") and (graphs != "LOGS")):
        fig = "NONE"
        
    print("The blue line is the initial balanced-growth path;")
    print("the orange line is the alternative balanced growth path;")
    print("the green line is the track of the economy as it transitions")
    print("from the baseline to the alternative BGP.")
    print(" ")
    
    print(n, "is the baseline labor-force growth rate")
    print(g, "is the baseline efficiency-of-labor growth rate")
    print(s, "is the baseline savings rate")
    print(" ")
          
    print(n + Delta_n, "is the alternative labor-force growth rate")
    print(g + Delta_g, "is the alternative efficiency-of-labor growth rate")
    print(s + Delta_s, "is the alternative savings-investment rate")
    print(" ")
    
    print(delta, "is the depreciation rate")
    print(alpha, "is the orientation-of-growth-toward-capital parameter")
    
    SGM_dict = {}
    SGM_dict["df"] = sg_df
    SGM_dict["plots"] = fig
    
    return SGM_dict

In [ ]:
# Now let's see how this works in a base case...

output = sgm_3_bgp_100yr_run(L0=1000, E0=1, graphs = "LEVELS")

In [ ]:
# Now let's see how this works in a base case and with the labor-
# force growth rate boosted by 1% per year in the alternative case...

output = sgm_3_bgp_100yr_run(L0=1000, E0=1, Delta_n=0.01)

In [ ]:
# Now let's save our results to a particular variable—output1—
# and look at what the value of the capital stock is in year 50...

# Save the output of fhe function...

output1 = sgm_3_bgp_100yr_run(L0=1000, E0=1, n=0.0, g=0.0, s=0.20, 
    alpha=0.5, delta=0.025, Delta_s=0.0, Delta_g=0, Delta_n=0.01, graphs="LEVELS")

# Now look at the 50th value of the "Capital" series in the
# "df" lobe of the dictionary "output1"...

print(" ")
print(" ")
print("The value of the capital stock in the 50th year =", output1["df"].Capital[50])

Now insert code cells between this markdown cell and the "# Task C answer variables" code cell. In them, run three simulations. in each simulation boost one of the savings rate, the efficiency of labor growth rate, and the labor force growth rate by one percentage point. 

Then calculate the values of K/Y in the 50th year of the simulation for each of your three simulations. In the "# Task C answer variables" code cell, set the variables equal to their values in your three respective simulation runs. In the "Task C Answer Description" markdown cell, write two paragraphs telling us what you find out about the effects of these different changes in the economic environment on a Solow growth model economy from these simulation runs:

In [ ]:
# Task C answer variables

KoverY_50_splus = __ # In the simulation run boosting the savings rate, the value of 
                     # the capital output ratio in the 50th year goes here

KoverY_50_nplus = __ # In the simulation run boosting the labor force growth rate, the
                     #  value of the capital output ratio in the 50th year goes here

KoverY_50_gplus = __ # In the simulation run boosting the efficiency of labor growth
                     # rate, the log value of the capital output ratio in the 50th year 
                     # goes here

<font color = "blue">**Task C Answer Discussion**:
    
</font>



In [ ]:
ok.grade('q01')

----

&nbsp;

**(Task D): Different Values of the Production Function Parameter $\alpha$**

Still working with our simulation function, insert code cells immediately below to calculate—logs and levels—the effect in the Solow growth model, starting from our default base case balanced growth path, of a permanent six percentage point increase in the savings-investment rate s for five different values of the production function curvature parameter alpha ($\alpha$): 1/4, 1/3, 1/2, 2/3, and 3/4. 

Also calculate the values of Y/L in the 50th year of the simulation for each of your  simulations. In the subsequent code cell "Task D answer variables" set the variables equal to their values in your five respective simulation runs.

In the subsequent markdown cell "Task D Answer Discussion", write two paragraphs about what these simulation runs tell you about the differences between economies in which diminishing returns to investment set in quickly because growth is oriented away from capital and investment ($ \alpha $, near zero), and economies in which diminishing returns set in slowly because growth is oriented toward capital and investment ($ \alpha $ high, near one):

In [ ]:
# Task D answer variables

YoverL_50_alphaquarter = __ # In the simulation run boosting the savings rate 
                            # with alpha=0.25, the value of output-per-worker
                            # in the 50th year goes here

YoverL_50_alphathird = __ # In the simulation run boosting the savings rate 
                          # with alpha=1/3, the value of output-per-worker
                          # in the 50th year goes here

YoverL_50_alphahalf = __ # In the simulation run boosting the savings rate 
                         # with alpha=0.5, the value of output-per-worker
                         # in the 50th year goes here

YoverL_50_alphatwothirds = __ # In the simulation run boosting the savings rate 
                              # with alpha=2/3, the value of output-per-worker
                              # in the 50th year goes here
        
YoverL_50_alphathreequarters = __ # In the simulation run boosting the savings rate 
                                  # with alpha=3/4, the value of output-per-worker
                                  # in the 50th year goes here


<font color="blue"> **Task D Answer Discussion**:
    
</font>

In [ ]:
ok.grade('q02')

----

&nbsp;

**(Task E): Calibrating the Model to the U.S. Today**

The code cell below gives parameter values roughly calibrated to the U.S. economy today, and sets two variables—KoverYstarinitial and YoverLstarinitial—equal to the capital/output ratio and output per worker level for the U.S. today along its current balanced growth path. Copy and paste the code cell, and alter it to calculate and print two variables:

    KoverYstaralt
    YoverLstaralt
    
equal to their alternative balanced growth path values if the U.S. savings-investment rate had been 2.75 percentage points higher. Then add more code cells calculating new variables:

    KoverYstarbase_F2
    YoverLstarbase_F2
    KoverYstaralt_F2
    YoverLstaralt_F2

corresponding to a Solow growth model economy with alpha = 1/2 and YoverLstarbase_F2 = 130000

In [ ]:
# in the U.S. today values are approximately:

n = 0.01
g = 0.015
delta = 0.025

sbase = 0.22
alpha = 0.333
 
KoverYstarbase = sbase/(n + g + delta)

E = 65067.5831863

YoverLstarbase = KoverYstarbase**(alpha/(1-alpha)) * E

print(KoverYstarbase, "= KoverYstarbase")
print(YoverLstarbase, "= YoverLstarbase")

In [ ]:
# higher savings-investment rate s:

n = 0.01
g = 0.015
delta = 0.025

sbase = 0.22
alpha = 0.333

salt = sbase + 0.0275
 
KoverYstaralt = salt/(n+g+delta)
YoverLstaralt = 

print(KoverYstaralt, "= KoverYstaralt")
print(YoverLstaralt, "= YoverLstaralt")

In [ ]:
# higher alpha and higher savings-investment rate s

n = 0.01
g = 0.015
delta = 0.025

sbase = 0.22
alpha = 0.333
 
KoverYstarbase_F2
YoverLstarbase_F2
KoverYstaralt_F2
YoverLstaralt_F2

print(KoverYstarbase_F2, "= KoverYstarbase_F2")
print(YoverLstarbase_F2, "= YoverLstarbase_F2")
print(KoverYstaralt_F2, "= KoverYstaralt_F2")
print(YoverLstaralt_F2, "= YoverLstaralt_F2")

In [ ]:
ok.grade('q05')

In [ ]:
_ = ok.submit()

## Appendix: Programming Dos and Don'ts...

### A Running List...

1. **Do** restart your kernel and run cells up to your current working point every fifteen minutes or so. Yes, it takes a little time. But if you don't, sooner or later the machine's namespace will get confused, and then you will get confused about the state of the machine's namespace, and by assuming things about it that are false you will lose hours and hours...   
&nbsp;   

2. **Do** edit code cells by copying them below your current version and then working on the copy: when you break everything in the current cell (as you will), you can then go back to the old cell and start fresh...   
&nbsp;

3. **Do** exercise agile development practices: if there is a line of code that you have not tested, test it. The best way to test is to ask the machine to echo back to you the thing you have just created in its namespace to make sure that it is what you want it to be...   
&nbsp;

4. **Do** take screenshots of your error messages...   
&nbsp;

5. **Do** google your error messages: ms. google is your best friend here...   
&nbsp;

6. **Do not** confuse assignment ("=") and test for equality ("=="). In general, if there is an "if" anywhere nearby, you should be testing for equality. If there is not, you should be assignment a variable in your namespace to a value. **Do** curse the mathematicians 500 years ago who did not realize that in the twenty-first century it would be very convenient if we had different and not confusable symbols for equals-as-assignment and equals-as-test...   
&nbsp;

----

&nbsp;

**(Task ∞) Programming Practices**

If it strikes you that anything should be added to this list of programming dos and don'ts, please email it to me at <delong@econ.berkeley.edu>